In [2]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import serial
import tdwf
import time
import os

#-[Configurazione seriale]-----------------------------------------------------
ser = serial.Serial()
ser.port = "COM8"
ser.baudrate = 115200
ser.timeout = 1
ser.rts = False
ser.dtr = False
ser.open()

def query(message, flagR=True, flagW=True):
    if flagW:    
        message += '\n'
        ser.write(message.encode('utf-8'))
    if flagR:
        res = ser.readline().decode('utf-8')
        return res


for i in range(12):
    response = ser.readline().decode('utf-8')
    print(response[:-1])


#-[Configurazione AD2]-------------------------------------------------------
ad2 = tdwf.AD2(iconfig=2)

ad2.vdd = +5
ad2.power(True)

#   2. Configurazione generatore di funzioni
wgen = tdwf.WaveGen(ad2.hdwf)
wgen.w1.config(ampl = 5, offs= 0, func=tdwf.funcSine, freq = 40968)
wgen.w1.start()

#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=1e6
scope.npt=int(16384)
scope.ch1.rng = 10
scope.ch2.rng = 10
scope.trig(True, level = 1, sour = tdwf.trigsrcCh1, delay = 0.04)


#-[Saving pipeline]-----------------------------------------------------
Directory = f"angular_{time.strftime('%Y%m%d_%H%M%S')}" # directory name based on the current date and time
parent_dir = "../../Data"
path = os.path.join(parent_dir, Directory) # creating the directory
os.mkdir(path)
print(f"Data will be saved in {path}\n")
log_file = os.path.join(path, f"log.txt")   #creating the log file
message = input("Please insert a \bMEANINGFUL\b comment to understand what the measurement in about: ")
with open(path+"/log.txt", "a") as log:
    log.write(time.strftime('%H%M%S') + ' >>> ' + message + "\n")
    log.close()



Digilent WaveForms SDK versione 3.22.2












Dispositivo #1 [SN:210321B1F3F8, hdwf=1] connesso!
Configurazione #2
Data will be saved in ../../Data\angular_20250515_121635



In [3]:
import numpy as np
rangeX = [0, 80000]
rangeY = [-400, 400]



translations = np.linspace(rangeX[0], rangeX[1], 11, dtype=int)
angules = np.linspace(rangeY[0], rangeY[1], 360, dtype=int)

print(angules)

#print(angules)



def setAngle(angle):
    print( query("ty="+str(angle))[:-1] )
    res = int(query("y?").split('=')[1])
    while res != angle:
        res = int(query("y?").split('=')[1])
    return res

def setX(x):
    print( query("tx="+str(x))[:-1] )
    res = int(query("x?").split('=')[1])
    while res != x:
        res = int(query("x?").split('=')[1])
    return res


setX(0)
setAngle(0)


for i in range(len(angules)):
    setAngle(angules[i])

#    res = query("ty="+str(angules[i]))
    # print(res[:-1])

    # make a measurement at the current angle in order to get the angular power distribution
    scope.sample()
    tt = scope.time.vals.tolist()
    tx = scope.ch1.vals.tolist()
    rx = scope.ch2.vals.tolist()

    info = f"Angular measurement - Y = {angules[i]:3.2f} \n"
    data_to_save = np.c_[tt, tx, rx]
    info = info + f"time [s]\tch1 [V]\tch2 [V]\n"

    filename = f"angular_{angules[i]:.0f}"
    np.savetxt(path+'/'+filename+".txt", data_to_save, delimiter="\t", header=info, fmt="%s")


break














res = query("tx="+str(rangeX[1]))
print(res[:-1])

posX = int(query("x?").split('=')[1])
first = True
time.sleep(.5)


while(posX < 0.98 * rangeX[1]):
    posX = int(query("x?").split('=')[1])
    print(posX)
scope.sample()
#    tt.append(scope.time.vals.tolist())
#    tx.append(scope.ch1.vals.tolist())
#    rx.append(scope.ch2.vals.tolist())


tt = tt + scope.time.vals.tolist()
tx = tx + scope.ch1.vals.tolist()
rx = rx + scope.ch2.vals.tolist()
res = query("tx=" + str(rangeX[0]))
print(res[:-1])

info = f"Doppler Shift measurement - X = {rangeX[0]:5.0f} -> {rangeX[1]:5.0f} - vX = {query("sx?").split('=')[1] } \n"

data_to_save = np.c_[tt, tx, rx]
info = info + f"time [s]\tch1 [V]\tch2 [V]\n"


filename = f"dopplerShift-{rangeX[0]:.0f}-{rangeX[1]:.0f}"

np.savetxt(path+'/'+filename+".txt", data_to_save, delimiter="\t", header=info, fmt="%s")



[-400 -398 -396 -394 -392 -389 -387 -385 -383 -380 -378 -376 -374 -372
 -369 -367 -365 -363 -360 -358 -356 -354 -351 -349 -347 -345 -343 -340
 -338 -336 -334 -331 -329 -327 -325 -323 -320 -318 -316 -314 -311 -309
 -307 -305 -302 -300 -298 -296 -294 -291 -289 -287 -285 -282 -280 -278
 -276 -273 -271 -269 -267 -265 -262 -260 -258 -256 -253 -251 -249 -247
 -245 -242 -240 -238 -236 -233 -231 -229 -227 -224 -222 -220 -218 -216
 -213 -211 -209 -207 -204 -202 -200 -198 -195 -193 -191 -189 -187 -184
 -182 -180 -178 -175 -173 -171 -169 -167 -164 -162 -160 -158 -155 -153
 -151 -149 -146 -144 -142 -140 -138 -135 -133 -131 -129 -126 -124 -122
 -120 -117 -115 -113 -111 -109 -106 -104 -102 -100  -97  -95  -93  -91
  -89  -86  -84  -82  -80  -77  -75  -73  -71  -68  -66  -64  -62  -60
  -57  -55  -53  -51  -48  -46  -44  -42  -39  -37  -35  -33  -31  -28
  -26  -24  -22  -19  -17  -15  -13  -11   -8   -6   -4   -2    1    3
    5    7   10   12   14   16   18   21   23   25   27   30   32   34
   36 

SyntaxError: 'break' outside loop (3310087864.py, line 55)

In [4]:
query(input(">> "))

'TY=0\r\n'

In [5]:
ser.close()